In [ ]:
import requests
import pymongo
import a1chemy.data_source as data_source
from a1chemy.common import Tag
from a1chemy.common import Tree

mongo_client = pymongo.MongoClient("mongodb://localhost:27017/", username='a1chemy', password='1B2C9046-E3CC-447F-9961-E125759BA44F')
mongo_ticks_client = data_source.MongoTicks(mongo_client=mongo_client)
mongo_tag_client = data_source.MongoTags(mongo_client=mongo_client)

In [ ]:
# #申万一级行业数据

# # 删除数据
# for k, v in mongo_tag_client.tree(id='swsindex').root.children.items():
#     mongo_tag_client.delete_by_parent(parent=k)
#     mongo_tag_client.delete(id=k)

# # 重建tags
# sw_sectors_root_tag = Tag(id='swsindex', parent=None, values={})
# mongo_tag_client.insert(tag=sw_sectors_root_tag)

# data_source.parse_sw_sectors_save_to_file('/Users/line/workspace/freedom/doc/SwClass.xls', 
#                          '/Users/line/workspace/freedom/A1chemy/data/sw_sector.json')
# sw_sectors_result = data_source.parse_sw_sectors(source='/Users/line/workspace/freedom/doc/SwClass.xls')
# for s in sw_sectors_result:
#     sector_tag = Tag(id=s['sector'], parent=sw_sectors_root_tag.id, values={})
#     mongo_tag_client.insert(tag=sector_tag)
#     id = s['exchange'] + '_' + s['symbol']
#     parent=sector_tag.id
#     values={'exchange': s['exchange'],'symbol': s['symbol']}
#     stock_tag = Tag(id=id, parent=parent,values=values)
#     mongo_tag_client.insert(tag=stock_tag)

In [ ]:
#沪深300
import requests
import pymongo
import a1chemy.data_source as data_source
from a1chemy.common import Tag
from a1chemy.common import Tree


def parse_sci_to_mongo(url, xls_path, json_file_path, index_tag_id):
    data_source.download_csi_xls(url=url, xls_path=xls_path)
    data_source.parse_csi_xls_file_and_write_to_file(source=xls_path, target=json_file_path)
    index_result = data_source.parse_csi_xls_file(source=xls_path)

    
    index_tag = Tag(id=index_tag_id, parent=None, values={})
    mongo_tag_client.insert(tag=index_tag)
    mongo_tag_client.delete_by_parent(parent=index_tag_id)
    for stock in index_result:
        stock_tag = Tag(
            id=stock['exchange'] + '_' + stock['symbol'],
            parent=index_tag.id,
            values={
                'exchange': stock['exchange'],
                'symbol': stock['symbol']
            }
        )
        mongo_tag_client.insert(tag=stock_tag)

parse_sci_to_mongo(url = 'https://csi-web-dev.oss-cn-shanghai-finance-1-pub.aliyuncs.com/static/html/csindex/public/uploads/file/autofile/cons/000300cons.xls',
                   xls_path='/Users/line/workspace/freedom/doc/000300cons.xls',
                   json_file_path = '/Users/line/workspace/freedom/A1chemy/data/csi_300.json',
                   index_tag_id = 'csi300')

parse_sci_to_mongo(url = 'https://csi-web-dev.oss-cn-shanghai-finance-1-pub.aliyuncs.com/static/html/csindex/public/uploads/file/autofile/cons/000905cons.xls',
                   xls_path='/Users/line/workspace/freedom/doc/000905cons.xls',
                   json_file_path = '/Users/line/workspace/freedom/A1chemy/data/csi_500.json',
                   index_tag_id = 'csi500')

parse_sci_to_mongo(url = 'https://csi-web-dev.oss-cn-shanghai-finance-1-pub.aliyuncs.com/static/html/csindex/public/uploads/file/autofile/cons/000906cons.xls',
                   xls_path='/Users/line/workspace/freedom/doc/000906cons.xls',
                   json_file_path = '/Users/line/workspace/freedom/A1chemy/data/csi_800.json',
                   index_tag_id = 'csi800')


#http://www.csindex.com.cn/zh-CN/indices/index-detail/000016
parse_sci_to_mongo(url = 'https://csi-web-dev.oss-cn-shanghai-finance-1-pub.aliyuncs.com/static/html/csindex/public/uploads/file/autofile/cons/000016cons.xls',
                   xls_path='/Users/line/workspace/freedom/doc/000016cons.xls',
                   json_file_path = '/Users/line/workspace/freedom/A1chemy/data/sse_50.json',
                   index_tag_id = 'sse50')

In [ ]:
#沪深300
import pymongo
import a1chemy.data_source as data_source
from a1chemy.common import Tag
from a1chemy.common import Tree

def parse_sci_sectors_to_mongo(url, xls_path, json_file_path, index_tag_id):
    data_source.download_csi_xls(url=url, xls_path=xls_path)
    data_source.parse_csi_xls_file_and_write_to_file(source=xls_path, target=json_file_path)
    index_result = data_source.parse_csi_xls_file(source=xls_path)

    
    index_tag = Tag(id=index_tag_id, parent='csi_300_sectors', values={})
    mongo_tag_client.insert(tag=index_tag)
    mongo_tag_client.delete_by_parent(parent=index_tag_id)
    for stock in index_result:
        stock_tag = Tag(
            id=stock['exchange'] + '_' + stock['symbol'],
            parent=index_tag.id,
            values={
                'exchange': stock['exchange'],
                'symbol': stock['symbol']
            }
        )
        mongo_tag_client.insert(tag=stock_tag)
        
sectors = (
    ('000914', '金融'),
    ('000912', '消费'),
    ('000910', '工业'),
    ('000915', '信息'),
    ('000911', '可选'),
    ('000913', '医药'),
    ('000909', '材料'),
    ('000908', '能源'),
    ('000916', '电信'),
    ('000917', '公用')
)
parent_tag = Tag(id='csi_sectors', parent=None, values={})
mongo_tag_client.insert(tag=parent_tag)
for sector in sectors:
    symbol = sector[0]
    tag_id = sector[1]
    parse_sci_sectors_to_mongo(url = 'https://csi-web-dev.oss-cn-shanghai-finance-1-pub.aliyuncs.com/static/html/csindex/public/uploads/file/autofile/cons/{}cons.xls'.format(symbol),
                   xls_path='/Users/line/workspace/freedom/doc/{}cons.xls'.format(symbol),
                   json_file_path = '/Users/line/workspace/freedom/A1chemy/data/csi_{}.json'.format(symbol),
                   index_tag_id = tag_id)

In [ ]:
## 行业分类Excel解析
import xlrd
## https://www.csindex.com.cn/zh-CN/indices/index-detail/000300#/dataService/industryClassification
def parse_csi_sectors_xls_file(source):
    data = xlrd.open_workbook(source)
    table = data.sheets()[0]
    result = []
    for i in range(1, table.nrows):
        row = table.row_values(i)
        exchange = None
        symbol = row[0]
        if 'HK' in symbol:
            exchange = 'HKEX'
            symbol = symbol.split('.')[0]
        elif symbol.startswith('6'):
            exchange = 'SH'
            symbol = exchange + symbol
        else:
            exchange = 'SZ'
            symbol = exchange + symbol
        d = {
            'exchange': exchange,
            'symbol': symbol,
            'name': row[1],
            'sector': row[3]
        }
        result.append(d)
    return result

result = parse_csi_sectors_xls_file('/Users/line/workspace/freedom/doc/行业分类.xlsx')

In [ ]:
sectors_map = {}
for d in result:
    sector = sectors_map.setdefault(d['sector'], [])
    sector.append(d)
mongo_tag_client.delete_by_parent(parent='csi_sectors')
mongo_tag_client.insert(tag=Tag(id='csi_sectors', parent=None, values={}))
for sector, stocks in sectors_map.items():
    index_tag = Tag(id=sector, parent='csi_sectors', values={})
    mongo_tag_client.insert(tag=index_tag)
    for stock in stocks:
        stock_tag = Tag(
            id=stock['exchange'] + '_' + stock['symbol'],
            parent=index_tag.id,
            values={
                'exchange': stock['exchange'],
                'symbol': stock['symbol']
            }
        )
        mongo_tag_client.insert(tag=stock_tag)

In [ ]:
sectors_map = {}
for d in result:
    sector = sectors_map.setdefault(d['sector'], [])
    sector.append(d)

for sector, stocks in sectors_map.items():
    print(sector)

In [ ]:
### tradingview上的行业分类
import a1chemy.data_source as data_source

tradingViewClient = data_source.TradingViewClient()
tradingViewClient.update_sectors(path='/Users/line/Downloads/china_2021-03-12.csv')